In [1]:
# Laden ont
from rdflib import Graph
from pathlib import Path
from rdflib import Graph, URIRef, RDF, Namespace, Literal
from rdflib.namespace import XSD
import pandas as pd
import os
from collections import namedtuple
# Initialisierung
OntologyData = namedtuple("OntologyData", [
    "ont_iri", "class_pr","op_pr","dp_pr","ont_path","g"
])

def reload_graph():
    ont_iri = "http://www.semanticweb.org/SkillOA/2025/3/KBI/"
    ont_ir = Namespace(ont_iri)
    class_pr = Namespace(f"{ont_iri}class_")
    op_pr = Namespace(f"{ont_iri}op_")
    dp_pr = Namespace(f"{ont_iri}dp_")

    ontology_path = Path(r"C:\Users\Alexander Verkhov\Downloads\KB1_v0.5.ttl")
    
    if not ontology_path.exists():
        raise FileNotFoundError(f"Datei nicht gefunden: {ontology_path}")

    g = Graph()

    # Statt file=..., den Inhalt als String lesen
    ttl_text = ontology_path.read_text(encoding="utf-8")
    g.parse(data=ttl_text, format="turtle")

    return OntologyData(ont_ir, class_pr, op_pr, dp_pr, ontology_path, g)
#reload_graph()
data = reload_graph()
print(data.ont_iri)

http://www.semanticweb.org/SkillOA/2025/3/KBI/


In [ ]:
#Berechning CI
from rdflib import Graph, RDF, RDFS, OWL, URIRef
import math

def get_all_classes(graph):
    return set(graph.subjects(RDF.type, OWL.Class))

def get_all_instances(graph):
    return set(graph.subjects(RDF.type, OWL.NamedIndividual))

def get_instances_of_class(graph, class_uri):
    return set(graph.subjects(RDF.type, URIRef(class_uri)))

def get_all_subclasses(graph, super_class):
    subclasses = {(super_class, 0)}  # Klasse selbst mit Tiefe 0
    to_visit = [(super_class, 0)]
    
    while to_visit:
        current, depth = to_visit.pop()
        for subclass in graph.subjects(RDFS.subClassOf, URIRef(current)):
            if (subclass, depth + 1) not in subclasses:
                subclasses.add((subclass, depth + 1))
                to_visit.append((subclass, depth + 1))
    
    return subclasses  # Set of (subclass_uri, depth)


def calculate_ci_for_class(graph, class_uri, total_instances):
    ci_value = 0.0
    subclasses = get_all_subclasses(graph, class_uri)

    for subclass_uri, depth in subclasses:
        instances = get_instances_of_class(graph, subclass_uri)
        if instances:
            print(f"{subclass_uri} mit {len(instances)} Instanzen auf Tiefe {depth}")
        ir = len(instances) / total_instances if total_instances > 0 else 0
        ci_value += ir / (2 ** depth)

    return ci_value

def calculate_ci_all_classes(graph):
    all_classes = get_all_classes(graph)
    all_instances = get_all_instances(graph)
    total_instances = len(all_instances)

    ci_results = {}

    for class_uri in all_classes:
        ci = calculate_ci_for_class(graph, class_uri, total_instances)
        ci_results[str(class_uri)] = ci

    return ci_results, total_instances


ci_per_class, total_instances = calculate_ci_all_classes(data.g)

for class_uri, ci in ci_per_class.items():
    print(f"CI({class_uri}) = {ci:.5f}")

average_ci = sum(ci_per_class.values()) / len(ci_per_class) if ci_per_class else 0
print(f"Durchschnittliches CI über alle Klassen: {average_ci:.5f}")


http://www.semanticweb.org/SkillOA/2025/3/KBI/class_Skill mit 11 Instanzen auf Tiefe 0
http://www.semanticweb.org/SkillOA/2025/3/KBI/class_ResourceModule mit 2 Instanzen auf Tiefe 0
http://www.semanticweb.org/SkillOA/2025/3/KBI/class_TechnicalProcess mit 1 Instanzen auf Tiefe 0
http://www.semanticweb.org/SkillOA/2025/3/KBI/class_Capability mit 3 Instanzen auf Tiefe 0
http://www.semanticweb.org/SkillOA/2025/3/KBI/class_ProposedCapability mit 3 Instanzen auf Tiefe 1
http://www.semanticweb.org/SkillOA/2025/3/KBI/class_ServiceRequest mit 2 Instanzen auf Tiefe 0
http://www.semanticweb.org/SkillOA/2025/3/KBI/class_TechnicalRestriction mit 10 Instanzen auf Tiefe 0
http://www.semanticweb.org/SkillOA/2025/3/KBI/class_FieldDevice mit 6 Instanzen auf Tiefe 0
http://www.semanticweb.org/SkillOA/2025/3/KBI/class_Resource mit 2 Instanzen auf Tiefe 0
http://www.semanticweb.org/SkillOA/2025/3/KBI/class_ProposedCapability mit 3 Instanzen auf Tiefe 0
http://www.semanticweb.org/SkillOA/2025/3/KBI/class_Sk

In [5]:
#Berechnung SPA
from rdflib import Graph, RDF, RDFS, OWL

def get_all_classes(graph):
    return set(graph.subjects(RDF.type, OWL.Class))

def get_properties_with_domains(graph):
    return set(graph.subjects(RDFS.domain, None))

def get_domain_of_property(graph, prop):
    return set(graph.objects(prop, RDFS.domain))

def get_superclasses(graph, cls):
    superclasses = set()
    to_visit = [cls]
    while to_visit:
        current = to_visit.pop()
        for super_cls in graph.objects(current, RDFS.subClassOf):
            if super_cls not in superclasses:
                superclasses.add(super_cls)
                to_visit.append(super_cls)
    return superclasses

def is_property_unique_to_subclass(graph, prop, subclass):
    domains = get_domain_of_property(graph, prop)
    for domain in domains:
        if domain == subclass:
            superclasses = get_superclasses(graph, subclass)
            for sup in superclasses:
                if (prop, RDFS.domain, sup) in graph:
                    return False
            return True
    return False

def calculate_spa_local(graph):
    classes = get_all_classes(graph)
    props = get_properties_with_domains(graph)

    unique_properties = set()

    for prop in props:
        for cls in classes:
            if is_property_unique_to_subclass(graph, prop, cls):
                unique_properties.add(prop)

    total_unique_properties = len(unique_properties)
    total_classes = len(classes)

    if total_classes == 0:
        return 0.0

    return total_unique_properties / total_classes

spa_value = calculate_spa_local(data.g)
print(f"SPA-Value: {spa_value:.2f}")

SPA-Value: 5.41


In [12]:
#Durchschnittliches SPI
import requests

endpoint_url = r"http://localhost:7200/repositories/FMEA3"

def send_query(query):
    response = requests.post(endpoint_url, data={"query": query}, headers={"Accept": "application/sparql-results+json"})
    if response.status_code == 200:
        return response.json()['results']['bindings']
    else:
        error_details = response.text 
        raise Exception(f"Query failed with status code {response.status_code}: {error_details}")

def get_classes_and_superclasses():
    query_classes = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>

    SELECT DISTINCT ?class ?superclass
    WHERE {
        ?class rdf:type owl:Class .
        OPTIONAL { ?class rdfs:subClassOf ?superclass }
        FILTER (isURI(?class) && isURI(?superclass))
    }
    """
    return send_query(query_classes)

def calculate_spi(class_uri, superclass_uri):
    query_total_triples = f"""
    SELECT (COUNT(*) AS ?count)
    WHERE {{
        ?instance a <{class_uri}> ;
                 ?property ?value .
    }}
    """
    total_triples = send_query(query_total_triples)
    total_count = sum(int(row['count']['value']) for row in total_triples)

    query_specific_triples = f"""
    SELECT (COUNT(*) AS ?count)
    WHERE {{
        ?instance a <{class_uri}> ;
                 ?property ?value .
        FILTER NOT EXISTS {{
            ?instance a <{superclass_uri}>;
                      ?property ?value .
        }}
    }}
    """
    specific_triples = send_query(query_specific_triples)
    specific_count = sum(int(row['count']['value']) for row in specific_triples)
    print(f"insgesamt: {total_count}")
    print(f"spezifisch: {specific_count}")
    
    
    return specific_count / total_count if total_count > 0 else 0

def calculate_average_spi():
    class_info = get_classes_and_superclasses()
    spi_values = []
    for class_data in class_info:
        class_uri = class_data['class']['value']
        superclass_uri = class_data['superclass']['value'] if 'superclass' in class_data else None
        if superclass_uri:  # SPI nur berechnen, wenn eine Oberklasse vorhanden ist
            spi_value = calculate_spi(class_uri, superclass_uri)
            spi_values.append(spi_value)

    if spi_values:
        average_spi = sum(spi_values) / len(spi_values)
    else:
        average_spi = 0
    return average_spi

average_spi = calculate_average_spi()
print(f"Durchschnittlicher SPI: {average_spi:.10f}")

insgesamt: 70
spezifisch: 0
insgesamt: 56
spezifisch: 0
insgesamt: 56
spezifisch: 0
insgesamt: 56
spezifisch: 0
insgesamt: 77
spezifisch: 0
insgesamt: 77
spezifisch: 0
insgesamt: 77
spezifisch: 0
insgesamt: 271
spezifisch: 0
insgesamt: 271
spezifisch: 0
insgesamt: 42
spezifisch: 0
insgesamt: 42
spezifisch: 0
insgesamt: 42
spezifisch: 0
insgesamt: 66
spezifisch: 0
insgesamt: 66
spezifisch: 0
insgesamt: 66
spezifisch: 0
insgesamt: 46
spezifisch: 0
insgesamt: 46
spezifisch: 0
insgesamt: 46
spezifisch: 0
insgesamt: 71
spezifisch: 0
insgesamt: 71
spezifisch: 0
insgesamt: 71
spezifisch: 0
insgesamt: 71
spezifisch: 0
insgesamt: 71
spezifisch: 0
insgesamt: 71
spezifisch: 0
insgesamt: 81
spezifisch: 0
insgesamt: 81
spezifisch: 0
insgesamt: 81
spezifisch: 0
insgesamt: 81
spezifisch: 0
insgesamt: 81
spezifisch: 0
insgesamt: 81
spezifisch: 0
insgesamt: 264
spezifisch: 0
insgesamt: 264
spezifisch: 0
insgesamt: 48
spezifisch: 0
insgesamt: 48
spezifisch: 0
insgesamt: 48
spezifisch: 0
insgesamt: 633
s

In [ ]:
#SPI
from rdflib import Graph, RDF, RDFS, OWL, URIRef

def get_all_classes_with_superclasses(graph):
    result = []
    for cls in graph.subjects(RDF.type, OWL.Class):
        for superclass in graph.objects(cls, RDFS.subClassOf):
            result.append((cls, superclass))
    return result

def get_instances_of_class(graph, class_uri):
    return set(graph.subjects(RDF.type, URIRef(class_uri)))

def get_instance_properties(graph, instance_uris):
    props = set()
    for inst in instance_uris:
        for pred in graph.predicates(subject=inst):
            if pred != RDF.type:
                props.add(pred)
    return props

def calculate_spi_for_class(graph, class_uri, superclass_uri):
    class_instances = get_instances_of_class(graph, class_uri)
    superclass_instances = get_instances_of_class(graph, superclass_uri)

    class_props = get_instance_properties(graph, class_instances)
    superclass_props = get_instance_properties(graph, superclass_instances)

    total_count = len(class_props)
    specific_count = len(class_props - superclass_props)

    if total_count == 0:
        return 0.0

    return specific_count / total_count

def calculate_average_spi_local(graph):
    spi_values = []
    class_pairs = get_all_classes_with_superclasses(graph)

    print("SPI-Werte je Klasse:\n")

    for class_uri, superclass_uri in class_pairs:
        spi = calculate_spi_for_class(graph, class_uri, superclass_uri)
        spi_values.append(spi)
        print(f"🔹 Klasse: {class_uri}")
        print(f"   ↳ Oberklasse: {superclass_uri}")
        print(f"   ↳ SPI: {spi:.5f}\n")

    if not spi_values:
        return 0.0

    return sum(spi_values) / len(spi_values)


average_spi = calculate_average_spi_local(data.g)
print(f"Durchschnittlicher SPI: {average_spi:.10f}")

SPI-Werte je Klasse:

🔹 Klasse: http://www.semanticweb.org/SkillOA/2025/3/KBI/class_Actor
   ↳ Oberklasse: http://www.semanticweb.org/SkillOA/2025/3/KBI/class_FieldDevice
   ↳ SPI: 0.00000

🔹 Klasse: http://www.semanticweb.org/SkillOA/2025/3/KBI/class_AtomicSkill
   ↳ Oberklasse: http://www.semanticweb.org/SkillOA/2025/3/KBI/class_Skill
   ↳ SPI: 0.00000

🔹 Klasse: http://www.semanticweb.org/SkillOA/2025/3/KBI/class_CombinedSkill
   ↳ Oberklasse: http://www.semanticweb.org/SkillOA/2025/3/KBI/class_Skill
   ↳ SPI: 0.00000

🔹 Klasse: http://www.semanticweb.org/SkillOA/2025/3/KBI/class_FunctionalFeatureParameter
   ↳ Oberklasse: http://www.semanticweb.org/SkillOA/2025/3/KBI/class_SkillParameter
   ↳ SPI: 0.00000

🔹 Klasse: http://www.semanticweb.org/SkillOA/2025/3/KBI/class_GeometricalRestriction
   ↳ Oberklasse: http://www.semanticweb.org/SkillOA/2025/3/KBI/class_TechnicalRestriction
   ↳ SPI: 0.00000

🔹 Klasse: http://www.semanticweb.org/SkillOA/2025/3/KBI/class_MaterialProcessingRestri

Anzahl der Klassen mit Individuen: 72
Anzahl der Klassen : 88
ICR: 0.8181818181818182


In [ ]:
#Berechnung ICR
from rdflib import Graph, RDF, OWL

def get_all_classes(graph):
    return set(graph.subjects(RDF.type, OWL.Class))

def get_classes_with_instances(graph):
    classes_with_instances = set()
    for instance in graph.subjects(RDF.type, OWL.NamedIndividual):
        for cls in graph.objects(instance, RDF.type):
            classes_with_instances.add(cls)
    return classes_with_instances

def calculate_icr(graph):
    all_classes = get_all_classes(graph)
    used_classes = get_classes_with_instances(graph)

    if not all_classes:
        return 0.0

    return len(used_classes & all_classes) / len(all_classes)


# ICR berechnen
icr_value = calculate_icr(data.g)

# Ausgabe
print(f"Anzahl der Klassen: {len(get_all_classes(data.g))}")
print(f"Klassen mit mindestens einer Instanz: {len(get_classes_with_instances(data.g))}")
print(f"ICR-Wert: {icr_value:.5f}")


Anzahl der Klassen: 22
Klassen mit mindestens einer Instanz: 12
ICR-Wert: 0.50000


In [20]:
#Berechnung IPR
import requests

endpoint_url = r"http://localhost:7200/repositories/FMEA3"

def send_query(query):

    response = requests.post(endpoint_url, data={"query": query}, headers={"Accept": "application/sparql-results+json"})
    if response.status_code == 200:
        return response.json()['results']['bindings']
    else:
        error_details = response.text  
        raise Exception(f"Query failed with status code {response.status_code}: {error_details}")

def calculate_property_counts():
 
    query_total_properties = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    SELECT DISTINCT ?property
    WHERE {
        { ?property rdf:type rdf:Property }
        UNION
        { ?property rdf:type owl:ObjectProperty }
        UNION
        { ?property rdf:type owl:DatatypeProperty }
    }
    """
    total_properties = send_query(query_total_properties)
    total_property_count = len(set(result['property']['value'] for result in total_properties))


    query_used_properties = """
    SELECT DISTINCT ?property
    WHERE {
        ?subject ?property ?object .
        FILTER (?property != rdf:type)  # rdf:type ausschließen, da es sehr häufig vorkommt
    }
    """
    used_properties = send_query(query_used_properties)
    used_property_count = len(set(result['property']['value'] for result in used_properties))

    return total_property_count, used_property_count

total_property_count, used_property_count = calculate_property_counts()
print(f"Total number of properties (N(P)): {total_property_count}")
print(f"Number of instantiated properties (N(IP)): {used_property_count}")

ipr = used_property_count / total_property_count if total_property_count > 0 else 0
print(f"Instantiated Property Ratio (IPR): {ipr:.4f}")

Total number of properties (N(P)): 406
Number of instantiated properties (N(IP)): 179
Instantiated Property Ratio (IPR): 0.4409


In [ ]:
#IPR
from rdflib import Graph, RDF, RDFS, OWL

def get_all_defined_properties(graph):
    props = set()
    for p in graph.subjects(RDF.type, RDF.Property):
        props.add(p)
    for p in graph.subjects(RDF.type, OWL.ObjectProperty):
        props.add(p)
    for p in graph.subjects(RDF.type, OWL.DatatypeProperty):
        props.add(p)
    return props

def get_all_used_properties(graph):
    used_props = set()
    for s, p, o in graph:
        if p != RDF.type:  # rdf:type ausschließen
            used_props.add(p)
    return used_props

def calculate_ipr(graph):
    defined_props = get_all_defined_properties(graph)
    used_props = get_all_used_properties(graph)

    if not defined_props:
        return 0.0

    return len(used_props & defined_props) / len(defined_props)



# IPR berechnen
ipr_value = calculate_ipr(data.g)

# Ausgabe
print(f"Definierte Properties (N(P)): {len(get_all_defined_properties(data.g))}")
print(f"Verwendete Properties (N(IP)): {len(get_all_used_properties(data.g))}")
print(f"Instantiated Property Ratio (IPR): {ipr_value:.4f}")



Definierte Properties (N(P)): 120
Verwendete Properties (N(IP)): 59
Instantiated Property Ratio (IPR): 0.3833


In [4]:
#IMI
import requests


endpoint_url = r"http://localhost:7200/repositories/FMEA3"

def send_query(query):
   
    response = requests.post(endpoint_url, data={"query": query}, headers={"Accept": "application/sparql-results+json"})
    if response.status_code == 200:
        return response.json()['results']['bindings']
    else:
        error_details = response.text  
        raise Exception(f"Query failed with status code {response.status_code}: {error_details}")

def calculate_imi():
    query_classes_superclasses = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>

    SELECT ?class (COUNT(DISTINCT ?superclass) AS ?nsup)
    WHERE {
        ?class rdf:type owl:Class.
        OPTIONAL {?class rdfs:subClassOf ?superclass. FILTER(?superclass != owl:Thing)}
    } GROUP BY ?class
    """
    results = send_query(query_classes_superclasses)

    total_superclasses = 0
    class_count = 0

    for row in results:
        nsup = int(row['nsup']['value'])
        class_uri = row['class']['value']
        print(f"Klasse: {class_uri}, Anzahl direkter Oberklassen: {nsup}")
        total_superclasses += nsup
        class_count += 1

    if class_count > 0:
        average_multiple_inheritance = total_superclasses / class_count
    else:
        average_multiple_inheritance = 0

    if average_multiple_inheritance > 0:
        imi = 1 / average_multiple_inheritance
    else:
        imi = 0

    print(f"(Nc): {class_count}")
    print(f"(Summe nsup(Ci)): {total_superclasses}")
    print(f"durchschnittliche Anzahl Oberklasse: {average_multiple_inheritance:.4f}")
    print(f"(IMI): {imi:.4f}")

calculate_imi()

Klasse: http://www.semanticweb.org/alexanderverkhov/ontologies/2023/11/FMEA#Person, Anzahl direkter Oberklassen: 1
Klasse: http://www.semanticweb.org/alexanderverkhov/ontologies/2023/11/FMEA#FMEAVermeidungsmassnahmeFehlerartKombination, Anzahl direkter Oberklassen: 3
Klasse: http://www.semanticweb.org/alexanderverkhov/ontologies/2023/11/FMEA#ANachFMEA, Anzahl direkter Oberklassen: 3
Klasse: http://www.semanticweb.org/alexanderverkhov/ontologies/2023/11/FMEA#RisikobewertungNachFMEA, Anzahl direkter Oberklassen: 2
Klasse: http://www.semanticweb.org/alexanderverkhov/ontologies/2023/11/FMEA#APNachFMEA, Anzahl direkter Oberklassen: 3
Klasse: http://www.semanticweb.org/alexanderverkhov/ontologies/2023/11/FMEA#AVorFMEA, Anzahl direkter Oberklassen: 3
Klasse: http://www.semanticweb.org/alexanderverkhov/ontologies/2023/11/FMEA#APVorFMEA, Anzahl direkter Oberklassen: 3
Klasse: http://www.semanticweb.org/alexanderverkhov/ontologies/2023/11/FMEA#BNachFMEA, Anzahl direkter Oberklassen: 3
Klasse: ht

In [ ]:
#IMI
from rdflib import Graph, RDF, RDFS, OWL

def calculate_imi_local(graph):
    classes = set(graph.subjects(RDF.type, OWL.Class))
    total_superclasses = 0
    class_count = 0

    for cls in classes:
        superclasses = set(graph.objects(cls, RDFS.subClassOf))
        # owl:Thing ausschließen
        filtered_superclasses = {sc for sc in superclasses if str(sc) != str(OWL.Thing)}
        nsup = len(filtered_superclasses)

        print(f"Klasse: {cls}, Anzahl direkter Oberklassen: {nsup}")
        total_superclasses += nsup
        class_count += 1

    if class_count == 0:
        return 0.0

    average_superclass_count = total_superclasses / class_count
    imi = 1 / average_superclass_count if average_superclass_count > 0 else 0.0

    print(f"Anzahl Klassen (Nc): {class_count}")
    print(f"Summe direkter Oberklassen: {total_superclasses}")
    print(f"Durchschnittliche Anzahl Oberklassen: {average_superclass_count:.4f}")
    print(f"IMI-Wert: {imi:.4f}")

    return imi
imi_value = calculate_imi_local(data.g)

Klasse: http://www.semanticweb.org/SkillOA/2025/3/KBI/class_GeometricalRestriction, Anzahl direkter Oberklassen: 1
Klasse: http://www.semanticweb.org/SkillOA/2025/3/KBI/class_AtomicSkill, Anzahl direkter Oberklassen: 1
Klasse: http://www.semanticweb.org/SkillOA/2025/3/KBI/class_Sensor, Anzahl direkter Oberklassen: 1
Klasse: http://www.semanticweb.org/SkillOA/2025/3/KBI/class_SelectedCapability, Anzahl direkter Oberklassen: 1
Klasse: http://www.semanticweb.org/SkillOA/2025/3/KBI/class_Actor, Anzahl direkter Oberklassen: 1
Klasse: http://www.semanticweb.org/SkillOA/2025/3/KBI/class_Skill, Anzahl direkter Oberklassen: 0
Klasse: http://www.semanticweb.org/SkillOA/2025/3/KBI/class_CombinedSkill, Anzahl direkter Oberklassen: 1
Klasse: http://www.semanticweb.org/SkillOA/2025/3/KBI/class_ResourceModule, Anzahl direkter Oberklassen: 0
Klasse: http://www.semanticweb.org/SkillOA/2025/3/KBI/class_TechnicalProcess, Anzahl direkter Oberklassen: 0
Klasse: http://www.semanticweb.org/SkillOA/2025/3/KBI

In [ ]:
#Abfrage Präsentation
import requests

endpoint_url = r"http://localhost:7200/repositories/FMEA3"

def send_query(query):
   
    response = requests.post(endpoint_url, data={"query": query}, headers={"Accept": "application/sparql-results+json"})
    if response.status_code == 200:
        return response.json()['results']['bindings']
    else:
        error_details = response.text  
        raise Exception(f"Query failed with status code {response.status_code}: {error_details}")

def calculate_imi():
    query_classes_superclasses = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>

    SELECT ?class
    WHERE {
        ?class rdf:type owl:Class.
        uu :hatSchwerwie}
    } GROUP BY ?class
    """
    results = send_query(query_classes_superclasses)

    total_superclasses = 0
    class_count = 0

    for row in results:
        nsup = int(row['nsup']['value'])
        class_uri = row['class']['value']
        print(f"Klasse: {class_uri}, Anzahl direkter Oberklassen: {nsup}")
        total_superclasses += nsup
        class_count += 1

    if class_count > 0:
        average_multiple_inheritance = total_superclasses / class_count
    else:
        average_multiple_inheritance = 0

    if average_multiple_inheritance > 0:
        imi = 1 / average_multiple_inheritance
    else:
        imi = 0

    print(f"(Nc): {class_count}")
    print(f"(Summe nsup(Ci)): {total_superclasses}")
    print(f"durchschnittliche Anzahl Oberklasse: {average_multiple_inheritance:.4f}")
    print(f"(IMI): {imi:.4f}")

calculate_imi()